In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import os
import glob
import shutil
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

In [ ]:
batch_size = 32

width = 128
height = 128
epochs = 40
NUM_TRAIN = 2000
NUM_TEST = 1000
dropout_rate = 0.2
input_shape = (height, width, 3)

In [ ]:
import cv2
img=cv2.imread('train/0/67.jpg')
print(img.shape)

(512, 512, 3)


# New Section

In [ ]:
import os
%cd /content
if not os.path.isdir("efficientnet_keras_transfer_learning"):
  !git clone https://github.com/Tony607/efficientnet_keras_transfer_learning
%cd efficientnet_keras_transfer_learning/

/content
/content/efficientnet_keras_transfer_learning


In [ ]:
# Options: EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3
# Higher the number, the more complex the model is.
from efficientnet import EfficientNetB0 as Net
from efficientnet import center_crop_and_resize, preprocess_input

Using TensorFlow backend.


In [ ]:
# loading pretrained conv base model
conv_base = Net(weights='imagenet', include_top=False, input_shape=(64,64,1))

In [ ]:
cd ..

/content


# New Section

In [ ]:
cd drive/My Drive/recognizance1

/content/drive/My Drive/recognizance1


In [ ]:


# The directory where we will
# store our smaller dataset
# base_dir = './data/dog_vs_cat_small'
# os.makedirs(base_dir, exist_ok=True)

# Directories for our training,
# validation and test splits
train_dir ='train/'
# os.makedirs(train_dir, exist_ok=True)
# validation_dir = os.path.join('/', 'validation')
# os.makedirs(validation_dir, exist_ok=True)
test_dir ='test/'

train_0_dir = 'train/0'
# os.makedirs(train_0_dir, exist_ok=True)
train_1_dir = 'train/1'
# os.makedirs(train_1_dir, exist_ok=True)
train_2_dir ='train/2'
# os.makedirs(train_2_dir, exist_ok=True)
train_3_dir = 'train/3'
# os.makedirs(train_3_dir, exist_ok=True)
train_4_dir = 'train/4'
# os.makedirs(train_4_dir, exist_ok=True)


# Directory with our validation dog pictures
test_0_dir = 'test/0'
# os.makedirs(validation_0_dir, exist_ok=True) 
test_1_dir = 'test/1'
# os.makedirs(validation_1_dir, exist_ok=True)
test_2_dir ='test/2'
# os.makedirs(validation_2_dir, exist_ok=True)
test_3_dir = 'test/3'
# os.makedirs(validation_3_dir, exist_ok=True)
test_4_dir ='test/4'
# os.makedirs(validation_4_dir, exist_ok=True)


In [ ]:
ls

sample.csv  test/  test1/  train/  train_label.csv


In [ ]:
print('total training 0 images:', len(os.listdir(train_0_dir)))
print('total training 1 images:', len(os.listdir(train_1_dir)))
print('total training 2 images:', len(os.listdir(train_2_dir)))
print('total training 3 images:', len(os.listdir(train_3_dir)))
print('total training 4 images:', len(os.listdir(train_4_dir)))


# print('total training 1 images:', len(os.listdir(train_1_dir)))
print('total validation 0 images:', len(os.listdir('test/0')))
print('total validation 1 images:', len(os.listdir('test/1')))

print('total validation 2 images:', len(os.listdir('test/2')))
print('total validation 3 images:', len(os.listdir('test/3')))

print('total validation 4 images:', len(os.listdir('test/4')))


total training 0 images: 314
total training 1 images: 120
total training 2 images: 428
total training 3 images: 279
total training 4 images: 278
total validation 0 images: 48
total validation 1 images: 24
total validation 2 images: 72
total validation 3 images: 48
total validation 4 images: 48


In [ ]:
height=64
width=64
batch_size=64
num_classes=5

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=180,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.2,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to target height and width.
        target_size=(height, width),
        color_mode='grayscale',
        batch_size=batch_size,
        # Since we use categorical_crossentropy loss, we need categorical labels
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='categorical')

Found 1419 images belonging to 5 classes.
Found 240 images belonging to 5 classes.


Efficient net

In [ ]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalMaxPooling2D(name="gap"))
# model.add(layers.Flatten(name="flatten"))

# model.add(layers.Dense(256, activation='relu', name="fc1"))
model.add(layers.Dense(512,activation='relu',name="extra_dense"))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(128,activation='relu',name="e_d_2"))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(num_classes, activation='softmax', name="fc_out"))

Own model

In [ ]:
model=models.Sequential()
model.add(layers.Conv2D(16,(5,5),padding='valid',activation='relu',input_shape=(64,64,1)))
model.add(layers.Conv2D(32,(5,5),padding='valid',activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(48,(3,3),activation='relu'))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(num_classes,activation='softmax'))


In [ ]:
model.summary()

In [ ]:
print('This is the number of trainable layers '
      'before freezing the conv base:', len(model.trainable_weights))

conv_base.trainable = False

print('This is the number of trainable layers '
      'after freezing the conv base:', len(model.trainable_weights))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      epochs=30,
      validation_data=validation_generator,
      verbose=1,
      use_multiprocessing=True,
      workers=4)

os.makedirs("./models", exist_ok=True)
model.save('./models/model01.h5')

!cp '/gdrive/My Drive/my_file' 'my_file'!cp '/gdrive/My Drive/my_file' 'my_file'!cp '/gdrive/My Drive/my_file' 'my_file'!cp '/gdrive/My Drive/my_file' 'my_file'!cp '/gdrive/My Drive/my_file' 'my_file'!cp '/gdrive/My Drive/my_file' 'my_file'!cp '/gdrive/My Drive/my_file' 'my_file'!cp '/gdrive/My Drive/my_file' 'my_file'!!!!!!!!!!!!!!!!!!!!!!!dc124cs1d15c1d591d!cassaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaadgrgrggrv 33ewsdaasqdwdadasasdsac1# New Section

In [ ]:
os.makedirs("./models", exist_ok=True)
model.save('./models/model01.h5')

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model.predict([x])[0][0]
    return result

